# 0. Upload the raw Dataset

In [1]:
import re
from utils.text_preprocessing import tokenize, stopwords
from index.prefix_tree import PrefixTree

[nltk_data] Downloading package punkt to /home/ekivo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ekivo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ekivo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ekivo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from parsers.russian_parser import Russian
rus_data = Russian()
rus_data.parse_data()
len(rus_data.dataset)

No diagnosis in  74308
No diagnosis in  74319


4682

# 1. Define resources
### Codes

In [3]:
codes = {
    '1':'полость',
    '2':'округлая тень',
    '3':'затемнение в легочной ткани',
    '4':'очаговые тени в легочной (более трех)',
    '5':'единичные очаговые тени в легочной ткани',
    '6':'ячеистость в легочной ткани',
    '7':'затемнение, увеличение корней (кроме фиброза)',
    '8':'фиброторакс',
    '9':'фиброз ограниченный (1-2 поля)',
    '10':'фиброз более двух легочных полей',
    '11':'фиброз одного легочного поля',
    '12':'фиброз корней выраженный',
    '13':'диффузный пневмосклероз',
    '14':'единичные фиброзные тяжи в легочной ткани',
    '15':'единичные тяжи в корнях',
    '16':'изменения в плевре (сросшееся плевра, наслоение, обызиствление)',
    '17':'междолевая "волосяная" плевра',
    '18':'деформации диафрагмы не связанные с плевральной патологией',
    '19':'состояние после операции',
    '20':'добавочная доля',
    '21':'изменение скелета грудной клетки (костные мозоли, синостозы, добавочные ребра, остеофиты и т.д.)',
    '22':'инородное тело, проецирующая на легочную ткань',
    '23':'обызиствление в легочной ткани (инодородное тело)',
    '24':'технический брак',
    '25':'прочее',
    '26':'обызиствление первичного комплекса',
    '27':'петрификаты крупные в легочной ткани',
    '28':'крупные петрификаты в корнях (диаметром не менее поперечного сечения ребра)',
    '29':'мелкие петрификаты в легочной ткани',
    '30':' множественные петрификаты (3 и более)',
    '31':'единичные петрификаты (1-2)',
    '32':'сомнительные петрификаты',
    '33':'множественные петрификаты в корнях (3 и более)',
    '34':'единичные петрификаты в корнях',
    '35':'сомнительные петрификаты в корнях',
    '36':'сердечно-сосудистая патология',
    '99':'норма'
    
}

### Vocabulary

In [4]:
voc = []
with open('resources/vocabulary.txt') as f:
    for line in f.readlines():
        voc.append(line[:len(line)-1])
print(voc)

['абсцедир', 'аксиальном', 'аксилярного', 'аорты', 'архивом', 'аталектаз', 'баз', 'без', 'ближе', 'бои', 'больше', 'больших', 'борозде', 'борозды', 'бронхопневмония', 'бронхоэктазы', 'бронхо', 'бронхо-сосудистый', 'бронхо-сосудистого', 'верхний', 'верхнедолевая', 'верхнее', 'верхне', 'верхне-долевая', 'верхние доли', 'верхняя доля', 'верхних долях', 'верхних отделах', 'верхних отделов', 'верхнее поле', 'верхних полях', 'верхней', 'верхнем', 'верхним', 'верхних', 'верхушек', 'верхушка', 'верхушках', 'верхушке', 'висцер', 'влево', 'воздух', 'воздуха', 'возможно', 'вправо', 'впроекции', 'всем', 'всю', 'выпот', 'выпуклый', 'выраженный', 'выраженно', 'гидропневмоторакс', 'гидроторакс', 'главного', 'головке', 'головка', 'гомогенного', 'грубые', 'грубых', 'грудного', 'деформация', 'деформирован', 'диафрагмы', 'диафрагмой', 'диафрагмальном', 'диафрагмального', 'динамика', 'динамики', 'динамикой', 'диссеминация', 'диссеминацией', 'дистальных', 'диффузная', 'диффузное', 'диффузный', 'добавочной'

### Russian stop words

In [5]:
stop_words = set(stopwords.words('russian'))

### Mistakes

In [6]:
mistakes = {}
with open('resources/undecided.txt', 'r') as f:
    for line in f.readlines():
        words = line.split('&')
        w1 = words[0]
        change = ''
        for i in range(1,len(words)):
            change += words[i]
        mistakes[w1[:len(w1)-1]] = change[1:len(change)-1]
mistakes

{'2ом': 'втором',
 '2ребра': 'второго ребра',
 '2стор': 'двухсторонняя',
 '5справа': '5 справа',
 '3р': 'третьего ребра',
 'аерхней': 'верхней',
 'брохопневмония': 'бронхопневмония',
 'бр/сос': 'бронхо-сосудистого',
 'верхгних': 'верхних',
 'верней': 'верхней',
 'вехней': 'верхней',
 'в-дол': 'верхнедолевая',
 'виброоза': 'xxx',
 'внрхних': 'верхних',
 'в/дол': 'верхне-долевая',
 'в/отд': 'верхнем отделе',
 'вн/дюле': 'в нижней доле',
 'во2': 'во втором',
 'во2-м': 'во втором',
 'во2м\\р': 'во втором межреберье',
 'в2-3': 'в двух или трех',
 'волн': 'волновой',
 'вплвральной': 'в плевральной',
 'впроекции': 'в проекции',
 'в1': 'в первом',
 'вс1': 'в с1',
 'вс-2': 'в с2',
 'вс9-10справа': 'в с9 и с10 справа',
 'вс3ах': 'в с3 xxx',
 'вс3справа': 'в с3 справа',
 'вс5справатень': 'в с5 справа тень',
 'вс8': 'в с8',
 'всредней': 'в средней',
 'всреднем': 'в средней',
 'выпукл': 'выпуклый',
 'выраж': 'выраженный',
 'диафр': 'диафрагмы',
 'динам': 'динамика',
 'диссем': 'диссеминация',
 'до3

In [7]:
def create_prefix_tree(data):
    pt = PrefixTree()
    for word in data:
        pt.add(word+"$")
    return pt

In [8]:
def symbol_to_end(string, symbol, cut_symbol=False):
            '''
            aaaSaaa -> aaaaaaS where S is special symbol
            '''
            index = string.find(symbol)
            return string[index + 1:] + string[:index]

In [9]:
prefix_tree = create_prefix_tree(voc)

# 2. Clean Up
### 2.1 Resolve questions

In [10]:
def resolve_questions(string):
    print(string)
    return string

In [11]:
pattern_question = re.compile(r'(?u)([\w/\\-]+)(\?)')
for report in rus_data.dataset:
    text = report.text.lower()
    #print(text)
    text = pattern_question.sub((lambda s:resolve_questions(s.group(1))),text)
    #print(text)
    #print()

диссеминация
3
20
саркоидоз
5/1
саркоидоз
3/1
5/4
30/4
5/1
зоб
слева
5/1
5
1
5/1
21/1
5/1
отр
пневмония
5/2
пневмония
тень
5/3
саркоидоз
5
пневмония
ключицей
слева
4\4
слева
30/1
21/1
2
5/1
пневмония
3/1
3/2-пневмония
-киста
тень
слева
саркоидоз
л-х
5/4
5/5
5/4
справа
саркоидоз
л-го
справа-инфильтрация
инфильтрация
5/1
21/5
кальцинаты
пневмония
пневмония
тени
3/1
саркоидоз
мтс
пневмония
пневмония
саркоидоз
5/1
тени
5/4
саркоидоз
саркоидоз
5/4
саркоидоз
пневмония
5
21
вглу
сосуда
2/2
21/2
саркоидоз
л-х
справа
саркоидоз
пневмония
слева
4
5
саркоидоз
3/3
пневмония
тени
справа
30
5/1
21/1
метастазы
2\4
инфильтрация
5/2
слева
инфильтрации
ссд
саркоидоз
рецидив
слева
мтс
пневмония
справа
5
слева
5/1
дин-ка
слева
пневмония
пневмония
саркоидоз
фиброз
пневмония
пневмония
5
30
3/2
справа
пневмония
дин-ка
пневмония
5/4
справа
фиброз
5/4
саркоидоз
плеврит
лгм
дин-ка
5/4
3/3
пневмония
пневмония
5/4


### 2.2 resolve missing space before 'норма'

In [11]:
def resolve_norm(prefix, norm):
    return prefix+" "+norm

### 2.2 Resolve word-word and word/word patterns¶

In [12]:
def resolve_reduction(before, word, after):
    # Check in mistakes
    #print(before+word+after)
    if before+word+after in mistakes:
        change = mistakes[before+word+after]
    else:
        query = after+'$'+before
        change = ''
        res = prefix_tree.find_all(query)
        if len(res)>0:
            for i in range(len(res)):
                res[i] = symbol_to_end(res[i],'$',True)
                #print(before+word+after,"->",res)
            change = res[0]
        else:
            query = before+word+after
            res = prefix_tree.find_all(query)
            if len(res)>0:
                for i in range(len(res)):
                    res[i] = symbol_to_end(res[i],'$',True)
                    #print(query,"->",res)
                change = res[0]
            else:
                shorters.add(before+word+after)
                change = before+word+after
    # print(before+word+after,"->",change)
    return change

In [13]:
def check_word(before,word, after):
    #print(word)
    if not word in voc and not word in stop_words:
        change = resolve_reduction("","",word)
    else:
        change = word
        # print(word,"->",change)
    #print("Change: ",change)
    return before+change+after

In [14]:
def fix_double_numeration(numbers, after):
    first = resolve_reduction("","",numbers[0]+"-м")
    second = resolve_reduction("","",numbers[1]+"-м")
    return first+" и "+second+after

In [15]:
def fix_numeration(number, after):
    change = resolve_reduction("","",number+"-м")
    return change+after

In [16]:
def fix_codes_loc(code, location, a,b):
    if int(code) > 0 and int(code) < 37 and code in codes:
        change = codes[code]
        a[0] += 1
        loc = resolve_reduction("","",location+"-м")
        return change+" в "+loc+" отделе"
    else:
        b[0] += 1
        return code

In [17]:
def fix_code(code, a,b):
    if int(code) > 10 and int(code) < 37 and code in codes:
        a[0] += 1
        return codes[code]
    
    else:
        b[0] += 1
        return code

In [18]:
def fix_double_c(one, two):
    return "c"+one+" и "+"c"+two

In [19]:
def fix_c(one):
    return "c"+one

In [20]:
def cals_c_s(one,two,a):
    a[0] += 1
    return one+two

In [21]:
codes_num = [0]
numbers_num = [0]
c_num = [0]

shorters = set()
pattern_norm = re.compile(r'(?u)(\w+)(норма)')
pattern_complex = re.compile(r'(?u)(\w+)(/|\\)(\w+)(-)(\w+)')
pattern_dash = re.compile(r'(?u)(\w+)(-)(\w+)')
pattern_slash = re.compile(r'(?u)(\w+)(/|\\)(\w+)')
pattern_words = re.compile(r'(?u)(\w+)')
pattert_double_num = re.compile(r'(?u)(\d)-(\d)(\s+межреберье|\s+межреберьях)')
pattern_numeration = re.compile(r'(?u)(\d)(\s+межреберье|\s+межреберьях)')
pattern_double_c = re.compile(r'(?u)(с )(\d+),(\d+)')
pattern_c = re.compile(r'(?u)(с )(\d+)')
pattern_c_num = re.compile(r'(?u)(с|s)(\d+)')
pattern_code_loc = re.compile(r'(?u)(\d+)(\\|/)(\d+)')
pattern_code = re.compile(r'(\d+)')
for report in rus_data.dataset:
    text = report.text.lower()
    if text == 'норма':
        continue
    print(text)
    text = pattern_norm.sub((lambda s:resolve_norm(s.group(1),s.group(2))),text)
   # print(text)
    text = pattern_complex.sub((lambda s:check_word("",s.group(1)+s.group(2)+s.group(3)+s.group(4)+s.group(5),"")),text)
    text = pattern_dash.sub((lambda s:resolve_reduction(s.group(1),s.group(2),s.group(3))),text)
    #print(text)
    text = pattern_slash.sub((lambda s:resolve_reduction(s.group(1),s.group(2),s.group(3))),text)
    #print(text)
    text = pattern_words.sub((lambda s:check_word("",s.group(1),"")),text)
    text = pattert_double_num.sub((lambda s:fix_double_numeration([s.group(1),s.group(2)], s.group(3))),text)
    text = pattern_numeration.sub((lambda s:fix_numeration(s.group(1),s.group(2))),text)
    text = pattern_double_c.sub((lambda s:fix_double_c(s.group(2),s.group(3))),text)
    text = pattern_c.sub((lambda s:fix_c(s.group(2))),text)
    text = pattern_code_loc.sub((lambda s:fix_codes_loc(s.group(1),s.group(3),codes_num,numbers_num)),text)
    text = pattern_code.sub((lambda s:fix_code(s.group(1),codes_num,numbers_num)),text)
    text = pattern_c_num.sub((lambda s:cals_c_s(s.group(1),s.group(2),c_num)),text)
    print(text)
    print()
    
shorters
print(codes_num)
print(numbers_num)
print(c_num)

слева в с1-2 инфильтрация л/ткани.рек: конс. фтизиатра.
слева в с1-2 инфильтрация легочной ткани.рекомендация: консультация. фтизиатра.

уч-к зат-я в сред отд-х л/поля справа+30/4
участок затенения в средний отделах легочного поля справа+ множественные петрификаты (3 и более) в четвертом отделе

35,15.образование над куполом диафрагмы справа.
сомнительные петрификаты в корнях,единичные тяжи в корнях.образование над куполом диафрагмы справа.

29\1-2 12\1-2
мелкие петрификаты в легочной ткани в первом отделе-2 фиброз корней выраженный в первом отделе-2

жидкость в плевральной полости слева
жидкость в плевральной полости слева

продолг тень во 2 м/реб справа
продолг тень во втором межреберье справа

инфильтрация вс3справа отганич.плеврой.
инфильтрация в с3 справа отганич.плеврой.

округлое образование в с4 слева.
округлое образование в с4 слева.

частичный аталектаз верхней доли правого легкого, жидкость в плевральной полости справа.расширен правый корень.
частичный аталектаз верхней доли

сморщено левое легкое(цирроз) на фоне множественные.полости.органы средостения в левой половине грудной полости.

21норма
изменение скелета грудной клетки (костные мозоли, синостозы, добавочные ребра, остеофиты и т.д.) норма

полиморфные очаги в верхней доле справа и верхней доле и прикорневой зоне слева.
полиморфные очаги в верхней доле справа и верхней доле и прикорневой зоне слева.

подозр на очаг тени ифибр тяжи в с1,2 с обеих сторон
подозрение на очаг тени и фиброзные тяжи в с1,2 с обеих сторон

справа в с 1 очаговые тени. рек: конс. фтизиатра ркпд.
справа в c1 очаговые тени. рекомендация: консультация. фтизиатра ркпд.

дополнительная округлая тень в s2 справа очаговая тень в 5 межр.справа
дополнительная округлая тень в s2 справа очаговая тень в 5 межреберий.справа

округлые метастатические образования в правом легком.
округлые метастатические образования в правом легком.

участок затен-я в с3 справа
участок затенения в с3 справа

затенение в сред доле справа
затенение в средний д

усиление и сгущение сосуд рисунка в первом и втором межреберье справа(пневмония?)

затенение в s3 слева аталектаз, жидкость в плевральной полости слева
затенение в s3 слева аталектаз, жидкость в плевральной полости слева

спонтанный пневмоторакс слева. частичный колабс левого легкого.
спонтанный пневмоторакс слева. частичный колабс левого легкого.

множественные очаговые тени на верхушке справа и в 1-2 межреберье
множественные очаговые тени на верхушке справа и в первом и втором межреберье

30/1  р+.
 множественные петрификаты (3 и более) в первом отделе  р+.

очаги на фоне фиброза в верхушке справа.
очаги на фоне фиброза в верхушке справа.

единичные очаговые тени на фоне фиброза в верхней доле левого легко.
единичные очаговые тени на фоне фиброза в верхней доле левого легкого.

образования в проекции с 4,5 правого легкого.
образования в проекции c4 и c5 правого легкого.

фокусная тень в сред.доле правого л-го.
фокусная тень в средний.доле правого левого.

остаточная инфильтрация по х

участок инфильтрация в зоне с3 справа изменение скелета грудной клетки (костные мозоли, синостозы, добавочные ребра, остеофиты и т.д.) в пятом отделе

в с1.2 обоих легких инфильтрация л/ткани.
в с1.2 обоих легких инфильтрация легочной ткани.

16/6 очаг тень во 2 м/реб справа
изменения в плевре (сросшееся плевра, наслоение, обызиствление) в шестом отделе очаг тень во втором межреберье справа

правостор пневмония.
правосторонний пневмония.

затенение средней доли правого легкого инфильтрация
затенение средней доли правого легкого инфильтрация

инфильтрация в проекции с5 слева с просветлением  в центре инф.
инфильтрация в проекции с5 слева с просветлением  в центре инфилтрация.

35  15  3,16/3,6
сомнительные петрификаты в корнях  единичные тяжи в корнях  3,изменения в плевре (сросшееся плевра, наслоение, обызиствление) в третьем отделе,6

21\4
изменение скелета грудной клетки (костные мозоли, синостозы, добавочные ребра, остеофиты и т.д.) в четвертом отделе

фокус инфильтрации в s8 справа